In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
import seaborn as sns
from cycler import cycler
from IPython.display import display
import datetime

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif

In [ ]:
data = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2022/data.csv")
sample_submission = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2022/sample_submission.csv")

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
data.head()

In [ ]:
sample_submission.head()

In [ ]:
data.info()

In [ ]:
float_feature=[f for f in data.columns if data[f].dtype=="float"]

In [ ]:
float_feature

In [ ]:
fig , ax = plt.subplots(4,4,figsize=(16,16)) 
for f, ax in zip(float_feature, ax.ravel()):
    ax.hist(data[f], density=True, bins=100)
    ax.set_title(f'Train {f}, std={data[f].std()}')
plt.suptitle('Histograms of the float features', y=0.93, fontsize=20)
plt.show()

In [ ]:
from pathlib import Path
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from tqdm import tqdm
import datetime

In [ ]:
SEED = 22
ESTIMATORS = 128

params = {'n_estimators': ESTIMATORS,
          'random_state': SEED,
          'tree_method' : 'gpu_hist'}

In [ ]:
features = data.columns.to_list()
data_completed = pd.DataFrame()


In [ ]:
np.where(data['F_1_0'].isnull())[0]

In [ ]:

for feat in tqdm(features):
    if data[feat].isnull().any():
        
        # Identify missing values...
        missing_values = list(np.where(data[feat].isnull())[0])
        not_missing_values = list(np.where(data[feat].isnull() == False)[0])
        
        trn_data = data.iloc[not_missing_values,]
        tst_data = data.iloc[missing_values,]
        
        X = trn_data.drop([feat,'row_id'],axis = 1)
        y = trn_data[feat]
        X_test = tst_data.drop([feat,'row_id'],axis = 1)
            
        model = XGBRegressor(**params)
        #model = LGBMRegressor(n_estimators = 10,metric = 'rmse')
        model.fit(X,y)
        
        y_predict = model.predict(X_test)
        imputed_data = data[feat]
        imputed_data.iloc[missing_values,] = y_predict
        
        data_completed = pd.concat([data_completed, imputed_data],axis = 1)
        
    else:
        data_completed = pd.concat([data_completed, data[feat]],axis = 1)

In [ ]:
sample_submission

In [ ]:
submission = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2022/sample_submission.csv', index_col='row-col')

In [ ]:
for i in tqdm(submission.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    submission.loc[i, 'value'] = data.loc[row, col]

submission.to_csv("gdbt_submission.csv")
submission

In [ ]:
submission.rename(columns={'index':'row-col'},inplace=True)
submission.to_csv("submission.csv")
submission